In [15]:
import numpy as np
import random
import pandas as pd
import gzip

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn import preprocessing

hour: keep hour num <br />
C1: maybe keep <br />
banner_pos: drop <br />
site_id: keep <br />
site_domain: keep <br />
site_category: maybe keep <br />
app_id: drop <br />
app_domain: drop <br />
app_category: drop <br />
device_id: drop <br />
device_ip: keep <br />
device_type: drop <br />
device_model: keep <br />
device_conn_type: keep <br />
C14 - C21 features: keep <br />

In [16]:
train_df_full = pd.read_csv("train.gz", compression='gzip', header='infer')

In [17]:
test_df = pd.read_csv("test.gz", compression='gzip', header='infer')

In [18]:
test_df.shape

(473610, 23)

In [19]:
train_df_full['click'].value_counts()

0    2141458
1     542329
Name: click, dtype: int64

In [20]:
train_df_full.drop(['banner_pos', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_type'], axis=1, inplace=True)
test_df.drop(['banner_pos', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_type'], axis=1, inplace=True)

In [21]:
def get_hour(hour):
    h = str(hour)[-2:]
    return int(h)

train_df_full['hour'] = train_df_full.hour.apply(get_hour)

In [31]:
test_df['hour'] = test_df.hour.apply(get_hour)

In [22]:
train_df_full['hour'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int64)

In [23]:
def convert_obj_to_int(self):
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self

In [24]:
train_df_full = convert_obj_to_int(train_df_full)

In [25]:
test_df = convert_obj_to_int(test_df)

In [28]:
train_df_full.drop("id", axis=1, inplace=True)

In [29]:
train_df_full

,click,hour,C1,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,site_id_int,site_domain_int,site_category_int,device_ip_int,device_model_int
0,0,0,5,2,15706,201,31,1300,0,3,-1,67,2536235637909526080,3462675629036857623,4028405133324091988,2061947887788174954,-739027035721191281
1,0,0,5,0,15704,201,31,1300,0,3,85,67,2536235637909526080,3462675629036857623,4028405133324091988,3926786909416420734,-2831400013463952790
2,0,0,5,0,15704,201,31,1300,0,3,85,67,2536235637909526080,3462675629036857623,4028405133324091988,8174721954748263756,-6214436056658734799
3,0,0,5,0,15706,201,31,1300,0,3,85,67,2536235637909526080,3462675629036857623,4028405133324091988,-8945280854665528184,-5921411830346609827
4,0,0,5,0,18993,201,31,1739,0,3,-1,145,2472638239944845512,7675038078679012993,-828405379516704027,-132786115088576729,-4562341497638536314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683782,0,1,5,0,20362,201,31,1911,0,7,-1,145,1529455106357640156,930569372652380983,-8005896523599973671,-6360544585565735748,5858475568989018710
2683783,0,1,5,0,20510,97,17,1929,3,131,-1,49,1529455106357640156,930569372652380983,-8005896523599973671,-8385909474522563117,-7892434950791851112
2683784,1,1,5,0,16208,201,31,1378,3,135,-1,11,-310564941995254762,-3117720592122035488,-1523113046048873988,7059975228367302776,-5594094824313543956
2683785,0,1,5,0,17262,201,31,1450,3,7,-1,11,1529455106357640156,930569372652380983,-8005896523599973671,-2962186408328843480,-3799675542112740494


In [32]:
test_df

,id,hour,C1,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,site_id_int,site_domain_int,site_category_int,device_ip_int,device_model_int
0,2683788,1,5,0,21694,201,31,2075,3,11,152,30,-3136566941393076338,6924731987684272180,6236696850212894917,-8661363667062954139,-2536958847997378218
1,2683789,1,5,0,16858,201,31,1465,3,7,201,11,-2021479305323603417,6313043768686837164,-8005896523599973671,-1360253000335087936,5513042897240158388
2,2683790,1,5,0,21759,201,31,2080,0,3,84,209,2536235637909526080,3462675629036857623,4028405133324091988,1787986909861005751,8399765227431840452
3,2683791,1,5,0,19950,201,31,1378,3,135,76,11,1120737337992626010,-3848951441382232459,-8005896523599973671,6403418337834804863,-428845057584110688
4,2683792,1,5,0,19950,201,31,1378,3,135,-1,11,8745081626891989089,2026797018831732987,-1523113046048873988,601674734043950273,-400373030669966121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473605,3157393,5,5,0,8330,201,31,339,3,143,78,11,-3955428746377681190,6902983613260277790,-1523113046048873988,-8177322625145931776,-400373030669966121
473606,3157394,5,5,0,6616,201,31,154,2,3,131,20,-1313984911697790376,-1404732400461587364,-8005896523599973671,-9127120872699028344,-6214436056658734799
473607,3157395,5,5,0,21763,201,31,2080,0,3,85,209,2536235637909526080,3462675629036857623,4028405133324091988,6350915645140383428,1755590103281616283
473608,3157396,5,5,0,15705,201,31,1300,0,3,-1,67,-5286137249468985085,4154470270322437269,-1523113046048873988,4499464705142657049,4399988844098000383


In [33]:
import lightgbm as lgb
X_train = train_df_full.loc[:, train_df_full.columns != 'click']
y_target = train_df_full.click.values
#create lightgbm dataset
msk = np.random.rand(len(X_train)) < 0.8
lgb_train = lgb.Dataset(X_train[msk], y_target[msk])
lgb_eval = lgb.Dataset(X_train[~msk], y_target[~msk], reference=lgb_train)

In [34]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves': 31, # defauly leaves(31) amount for each tree
    'learning_rate': 0.08,
    'feature_fraction': 0.7, # will select 70% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval,
                early_stopping_rounds=1000)

Start training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.063962 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's binary_logloss: 0.497758
Training until validation scores don't improve for 1000 rounds
[2]	valid_0's binary_logloss: 0.492654
[3]	valid_0's binary_logloss: 0.488332
[4]	valid_0's binary_logloss: 0.484582
[5]	valid_0's binary_logloss: 0.481111
[6]	valid_0's binary_logloss: 0.478301
[7]	valid_0's binary_logloss: 0.475683
[8]	valid_0's binary_logloss: 0.473568
[9]	valid_0's binary_logloss: 0.471641
[10]	valid_0's binary_logloss: 0.470042
[11]	valid_0's binary_logloss: 0.468364
[12]	valid_0's binary_logloss: 0.466687
[13]	valid_0's binary_logloss: 0.465226
[14]	valid_0's binary_logloss: 0.464188
[15]	valid_0's binary_logloss: 0.463237
[16]	valid_0's binary_logloss: 0.462235
[17]	valid_0's binary_logloss: 0.461122
[18]	valid_0's bina

In [35]:
X_test = test_df.loc[:, test_df.columns != 'id']

In [37]:
ypred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [38]:
output_dic_rfclassifier = {"id": list(test_df["id"]), "ctr": ypred}
output_df_rfclassifier = pd.DataFrame.from_dict(output_dic_rfclassifier)

In [39]:
output_df_rfclassifier

,id,ctr
0,2683788,0.001379
1,2683789,0.021815
2,2683790,0.221522
3,2683791,0.069631
4,2683792,0.138565
...,...,...
473605,3157393,0.073933
473606,3157394,0.075622
473607,3157395,0.266978
473608,3157396,0.153960


In [40]:
output_df_rfclassifier.to_csv("submission_v12.csv", index=False)